## Climate baseline

In [1]:
import os
import glob
import xarray as xr
import pandas as pd
import geopandas as gpd
import xesmf as xe
import regionmask
import xarray_regrid # type: ignore
# import netcdf4 

# when using "rename", dont change the target variable names (of course, change the original variable names according your files)

ModuleNotFoundError: No module named 'ESMF'

In [2]:
# the period
# period = slice("1980-01-01", "2019-12-31")
period = slice("1980-01-01", "2021-12-31")
# data now goes until 2021 so I want to include it... but also it goes back to 1960 - why not include it all? 
# or at the very least go back to the 1974 IGM DEM 

In [3]:
# sample file (to be shared)
# example    = xr.open_dataset("example_nc.nc")
example    = xr.open_dataset("/Users/milliespencer/Desktop/oggm_redo/rodrigo_data/histalp_merged_hef.nc")

# your DEM close to year 2000
# dem        = xr.open_dataset("file_dem.tif").rename({'y':'lat', 'x':'lon', 'band_data':'hgt'})
dem        = xr.open_dataset("/Users/milliespencer/Desktop/oggm_redo/rodrigo_data/srtm_dem_s37_w072_1arc_v3.tif").rename({'y':'lat', 'x':'lon', 'band_data':'hgt'})
# ^ my SRTM 2000 DEM for the area 
dem        = dem.sel(band=1, drop=True).drop("spatial_ref").fillna(0)
dem.hgt.attrs['units'] = "m"


/var/folders/3j/6dy_9gxj7vvgct178jkkp1680000gn/T/ipykernel_59421/2043866637.py:9: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  dem        = dem.sel(band=1, drop=True).drop("spatial_ref").fillna(0)


In [4]:
# Create path to monthly cr2met data (stored on my desktop)
pp_path = '/Users/milliespencer/Desktop/CR2MET_gridded_climate/pr/'
# within folder select each file 
pp_files = []
for file in os.listdir(pp_path):
    if file.endswith(".nc"):
        pp_files.append(file)

# " but for temperature 
t2m_path = '/Users/milliespencer/Desktop/CR2MET_gridded_climate/txn/'
t2m_files = []
for file in os.listdir('/Users/milliespencer/Desktop/CR2MET_gridded_climate/txn'):
    if file.endswith(".nc"):
        t2m_files.append(file)

pp_files = sorted(pp_files)
t2m_files = sorted(t2m_files)
# if you want to double check that it sorted properly: 
# for i in range(10):
#     print(pp_files[i])
#     print(txn_files[i])

In [5]:
pp_cr2met   = xr.open_dataset(pp_path + pp_files[0],  chunks = "auto").sel(time = period)
t2m_cr2met   = xr.open_dataset(t2m_path + t2m_files[0],  chunks = "auto").sel(time = period)


In [8]:
# print(t2m_cr2met.data_vars)
# shows there's tmin and tmax, I want tmean so I'll average the two 
# compute the average temperature
t2m_cr2met['tavg'] = (t2m_cr2met['tmin'] + t2m_cr2met['tmax']) / 2

# Print the new dataset with the added average temperature variable
print(t2m_cr2met)

<xarray.Dataset> Size: 712kB
Dimensions:  (lon: 220, lat: 800, time: 0)
Coordinates:
  * lon      (lon) float64 2kB -76.97 -76.92 -76.88 ... -66.12 -66.08 -66.03
  * lat      (lat) float64 6kB -56.98 -56.93 -56.88 ... -17.12 -17.07 -17.02
  * time     (time) datetime64[ns] 0B 
Data variables:
    tmin     (time, lat, lon) float64 0B dask.array<chunksize=(0, 800, 220), meta=np.ndarray>
    tmax     (time, lat, lon) float64 0B dask.array<chunksize=(0, 800, 220), meta=np.ndarray>
    cl_mask  (lat, lon) float32 704kB dask.array<chunksize=(800, 220), meta=np.ndarray>
    tavg     (time, lat, lon) float64 0B dask.array<chunksize=(0, 800, 220), meta=np.ndarray>
Attributes: (12/21)
    creation_date:          07-Sep-2022 14:35:39
    av_ens_qtl:             0.0
    agg_metr:               median
    authors:                Boisier et al.
    obs_file:               MERGE5_QC_tmin_day_1979_2021.csv
    topo_vars:              z
    ...                     ...
    par_ele_spl_rdiff_max:  0.2
  

In [9]:
# regrid and merge
regridder   = xe.Regridder(dem, pp_cr2met, "bilinear")
dem_cr2met  = regridder(dem)

# merge all variables
# rodrigo's code sayd pp:prcp but I think the variable is pr, not pp 
cr2met = xr.merge([pp_cr2met, t2m_cr2met, dem_cr2met]).rename({'pr':'prcp', 'tavg':'temp'})
cr2met = cr2met.where((cr2met.lon >= -73) & (cr2met.lon <= -71) & (cr2met.lat >= -38) & (cr2met.lat <= -36), drop=True)
# I just approximated the rectangle surrounding Nevados de Chillán to get the coordinates, may want to fine tune 
cr2met = cr2met[["lon", "lat", "time", "prcp", "temp", "hgt"]]

In [10]:
# unit attribute
cr2met.prcp.attrs['units'] = "mm"
cr2met.temp.attrs['units'] = "degc"
cr2met.hgt.attrs['units'] = "m"

# standard_name attribute
cr2met.prcp.attrs['standard_name'] = "precipitation"
cr2met.temp.attrs['standard_name'] = "temperature"

In [11]:
# save me
cr2met.to_netcdf("climate_to_use.nc")